In [0]:
import pandas as pd
import nltk
import spotipy
import spotipy.util as util
import json

In [0]:
key_file = 'keys.json'
keys = json.load(open(key_file))

SPOTIPY_CLIENT_ID= keys["client_id"]
SPOTIPY_CLIENT_SECRET= keys["client_secret"]
SPOTIPY_REDIRECT_URI= keys["redirect_url"]

In [0]:
token = util.prompt_for_user_token("1274418332", 
                                   scope = "user-library-read", 
                                   client_id=SPOTIPY_CLIENT_ID, 
                                   client_secret=SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri = SPOTIPY_REDIRECT_URI)
spotify = spotipy.Spotify(auth=token)

In [4]:
billboard = pd.read_csv("billboard_lyrics_1964-2015.csv")
billboard.head()

,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0


In [6]:
temp = billboard.sort_values('Year',ascending=False)
temp.head()

,Rank,Song,Artist,Year,Lyrics,Source
5099,100,how deep is your love,calvin harris and disciples,2015,i want you to breathe me in let me be your ai...,1.0
5036,37,talking body,tove lo,2015,bed stay in bed the feeling of your skin lock...,1.0
5026,27,good for you,selena gomez featuring aap rocky,2015,im on my 14 carats im 14 carat doing it up li...,1.0
5027,28,all about that bass,meghan trainor,2015,because you know im all about that bass bout ...,1.0
5028,29,style,taylor swift,2015,midnight you come and pick me up no headlight...,1.0


In [10]:
song = spotify.search('how deep is your love calvin harris and disciples')
song['tracks']['items'][0]

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7CajNmpbOovFoOoasH2HaY'},
    'href': 'https://api.spotify.com/v1/artists/7CajNmpbOovFoOoasH2HaY',
    'id': '7CajNmpbOovFoOoasH2HaY',
    'name': 'Calvin Harris',
    'type': 'artist',
    'uri': 'spotify:artist:7CajNmpbOovFoOoasH2HaY'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/5EehXjjMktLuJmbRsM7YfB'},
    'href': 'https://api.spotify.com/v1/artists/5EehXjjMktLuJmbRsM7YfB',
    'id': '5EehXjjMktLuJmbRsM7YfB',
    'name': 'Disciples',
    'type': 'artist',
    'uri': 'spotify:artist:5EehXjjMktLuJmbRsM7YfB'}],
  'available_markets': ['AD',
   'AR',
   'AT',
   'AU',
   'BE',
   'BG',
   'BO',
   'BR',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'EC',
   'EE',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HU',
   'ID',
   'IE',
   'IL',
   'IS',
   'IT',
   'JP',
   'LI',
  